# Access External Location

In [0]:
# here user_ext is extrnal locaion name
users_data = (
    spark.sql("DESCRIBE EXTERNAL LOCATION 'users_ext'").select("url").collect()[0][0]
)
print(users_data)

spark.read.csv(path=users_data, header=true, inferSchema=True).display()

## External location is not there so loading from volume

In [0]:
dfUser = spark.read.csv(
    path="/Volumes/dwlabdatabrickstest96_3538632722556102/quickstart_schema/sandbox/dataset/user_dataset/users_001.csv",
    header=True,
    inferSchema=True
)

# Create Function

In [0]:
%sql

CREATE FUNCTION dwlabdatabrickstest96_3538632722556102.quickstart_schema.full_name_udf(
  first_name STRING,
  last_name STRING
)
return first_name + ' ' + last_name

In [0]:
from pyspark.sql.functions import col
 
dfUserSelcted = dfUser.select(
    col("name").alias("first_name"), col("country").alias("last_name")
)
dfUserSelcted.createOrReplaceTempView("users_vw")

In [0]:
%sql
SELECT dwlabdatabrickstest96_3538632722556102.quickstart_schema.full_name_udf(first_name, last_name) from users_vw

## Apply Same UDF with Pyspark SQL

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def full_name_udf_second(f_name, l_name):
    return f_name + ' ' + l_name
 
joinNameUdf = udf(full_name_udf_second)

df_full_name = dfUser.select(
    joinNameUdf("first_name", "last_name").alias("full_name")
)
 
display(df_full_name)